In [69]:
import numpy as np

In [70]:
class Grid:
    def __init__(self, width, height, start):
        self.width = width
        self.height = height
        self.i = start[0]
        self.j = start[1]
        
    def set(self, rewards, actions):
        self.rewards = rewards
        self.actions = actions
        
    def set_state(self, s):
        self.i = s[0]
        self.j = s[1]
    
    def current_state(self):
        return (self.i, self.j)
    
    def is_terminal(self, s):
        return s not in self.actions
    
    def move(self, action):
        if action in self.actions[self.current_state()]:
            if action == 'U':
                self.i -= 1
            if action == 'D':
                self.i += 1
            if action == 'L':
                self.j -= 1
            if action == 'R':
                self.j += 1
                
        return self.rewards.get(self.current_state(), 0)
    
    def undo_move(self, action):
        if action == 'U':
            self.i += 1
        if action == 'D':
            self.i -= 1
        if action == 'L':
            self.j += 1
        if action == 'R':
            self.j -= 1
            
        assert (self.current_state() in self.all_states())
        
    def game_over(self):
        return self.is_terminal(self.current_state())
    
    def all_states(self):
        return set(self.actions.keys()).union(set(self.rewards.keys()))

In [71]:
def standard_grid():
    g = Grid(3, 4, (2, 0))
    rewards = {(0, 3) : 1, (1, 3) : -1}
    actions = {
        (0, 0) : ('D', 'R'),
        (0, 1) : ('L', 'R'),
        (0, 2) : ('L', 'D', 'R'),
        (1, 0) : ('U', 'D'),
        (1, 2) : ('U', 'D', 'R'),
        (2, 0) : ('U', 'R'),
        (2, 1) : ('L', 'R'),
        (2, 2) : ('L', 'R', 'U'),
        (2, 3) : ('L', 'U'),
    }
    
    g.set(rewards, actions)
    
    return g

In [72]:
def negative_grid(step_cost = -0.1):
    g = standard_grid()
    g.rewards.update({
        (0, 0) : step_cost,
        (0, 1) : step_cost,
        (0, 2) : step_cost,
        (1, 0) : step_cost,
        (1, 2) : step_cost,
        (2, 0) : step_cost,
        (2, 1) : step_cost,
        (2, 2) : step_cost,
        (2, 3) : step_cost,
    })
    
    return g

In [73]:
def play_game(agent, env):
    pass

======================== iterative_policy_evaluation =================================

In [74]:
SMALL_ENOUGH = 10e-4

In [83]:
def print_values(V, g):
    for i in range(g.width):
        print("----------------------")
        for j in range(g.height):
            v = V.get((i, j), 0)
            if v >= 0:
                print("{0:.2f}".format(v), "|", end="")
            else:
                print("{0:.2f}".format(v), "|", end="")
        print("")
        
def print_policy(P, g):
    for i in range(g.width):
        print("----------------------")
        for j in range(g.height):
            a = P.get((i, j), ' ')
            print(" ", a, ' ', end="")
        print("")

In [84]:
grid = standard_grid()
states = grid.all_states()

V = {}

for s in states:
    V[s] = 0
gamma = 1.0

while True:
    biggest_change = 0
    for s in states:
        old_v = V[s]
        
        if s in grid.actions:
            new_v = 0
            p_a = 1.0 / len(grid.actions[s])
            for a in grid.actions[s]:
                grid.set_state(s)
                r = grid.move(a)
                new_v += p_a * (r + gamma * V[grid.current_state()])
                grid.undo_move(a)
            V[s] = new_v
            biggest_change = max(biggest_change, np.abs(V[s] - old_v))
    if biggest_change < SMALL_ENOUGH:
        break
    print("Values for uniformly action actions:")
    print_values(V, grid)
    print("\n\n")

Values for uniformly action actions:
----------------------
0.00 |0.00 |0.22 |0.00 |
----------------------
0.00 |0.00 |-0.33 |0.00 |
----------------------
0.00 |0.00 |-0.28 |-0.50 |



Values for uniformly action actions:
----------------------
0.06 |0.11 |0.25 |0.00 |
----------------------
-0.01 |0.00 |-0.35 |0.00 |
----------------------
-0.07 |-0.14 |-0.38 |-0.64 |



Values for uniformly action actions:
----------------------
0.07 |0.15 |0.26 |0.00 |
----------------------
-0.02 |0.00 |-0.37 |0.00 |
----------------------
-0.11 |-0.22 |-0.43 |-0.69 |



Values for uniformly action actions:
----------------------
0.07 |0.17 |0.26 |0.00 |
----------------------
-0.04 |0.00 |-0.39 |0.00 |
----------------------
-0.15 |-0.27 |-0.46 |-0.71 |



Values for uniformly action actions:
----------------------
0.06 |0.17 |0.26 |0.00 |
----------------------
-0.05 |0.00 |-0.40 |0.00 |
----------------------
-0.17 |-0.30 |-0.48 |-0.73 |



Values for uniformly action actions:
----------------

========================= iterative policy evaluation ========================

In [86]:
GAMMA = 0.9
ALL_POSSIBLE_ACTIONS = ('U', 'D', 'L', 'R')
grid = negative_grid()

print("\nRewards:")
print_values(grid.rewards, grid)

policy = {}
for s in grid.actions.keys():
    policy[s] = np.random.choice(ALL_POSSIBLE_ACTIONS)
    
print("\nInitial policy:")
print_policy(policy, grid)

V = {}
states = grid.all_states()
for s in states:
    if s in grid.actions:
        V[s] = np.random.random()
    else:
        V[s] = 0

cnt = 0
while True:
    print("Iteration ", cnt)
    cnt += 1
    while True:
        biggest_change = 0
        for s in states:
            old_v = V[s]
            
            if s in policy:
                a = policy[s]
                grid.set_state(s)
                r = grid.move(a)
                V[s] = r + GAMMA * V[grid.current_state()]
                biggest_change = max(biggest_change, np.abs(old_v - V[s]))
                
        if biggest_change < SMALL_ENOUGH:
            break
                
    is_policy_converged = True
    for s in states:
        if s in policy:
            old_a = policy[s]
            new_a = None
            best_value = float('-inf')
            for a in ALL_POSSIBLE_ACTIONS:
                grid.set_state(s)
                r = grid.move(a)
                v = r + GAMMA * V[grid.current_state()]
                
                if v > best_value:
                    best_value = v
                    new_a = a
            policy[s] = new_a
            if new_a != old_a:
                is_policy_converged = False
                
    if is_policy_converged:
        break
            
print("\nvalues:")
print_values(V, grid)
    
print("\npolicy:")
print_policy(policy, grid)


Rewards:
----------------------
-0.10 |-0.10 |-0.10 |1.00 |
----------------------
-0.10 |0.00 |-0.10 |-1.00 |
----------------------
-0.10 |-0.10 |-0.10 |-0.10 |

Initial policy:
----------------------
  R    D    U       
----------------------
  D         U       
----------------------
  U    U    L    R  
Iteration  0
Iteration  1
Iteration  2

values:
----------------------
0.62 |0.80 |1.00 |0.00 |
----------------------
0.46 |0.00 |0.80 |0.00 |
----------------------
0.31 |0.46 |0.62 |0.46 |

policy:
----------------------
  R    R    R       
----------------------
  U         U       
----------------------
  U    R    U    L  


========================= value iteration ==================================

In [92]:
GAMMA = 0.9
ALL_POSSIBLE_ACTIONS = ('U', 'D', 'L', 'R')
grid = negative_grid()

print("\nRewards:")
print_values(grid.rewards, grid)

policy = {}
for s in grid.actions.keys():
    policy[s] = np.random.choice(ALL_POSSIBLE_ACTIONS)
    
print("\nInitial policy:")
print_policy(policy, grid)

V = {}
states = grid.all_states()
for s in states:
    if s in grid.actions:
        V[s] = np.random.random()
    else:
        V[s] = 0
        
while True:
    max_delta = 0
    
    for s in states:
        old_v = V[s]
        if s in policy:
            new_v = float('-inf')
            for a in ALL_POSSIBLE_ACTIONS:
                grid.set_state(s)
                r = grid.move(a)
                v = r + gamma * V[grid.current_state()]
                new_v = max(new_v, v)
            
            V[s] = new_v
            max_delta = max(max_delta, np.abs(old_v - V[s]))
        
    if max_delta < SMALL_ENOUGH:
        break

for s in policy.keys():
    best_a = None
    best_value = float('-inf')
    for a in ALL_POSSIBLE_ACTIONS:
        grid.set_state(s)
        r = grid.move(a)
        value = r + gamma * V[grid.current_state()]
        if best_value < value:
            best_value = value
            best_a = a
    policy[s] = best_a
        
print("\nvalues:")
print_values(V, grid)
    
print("\npolicy:")
print_policy(policy, grid)


Rewards:
----------------------
-0.10 |-0.10 |-0.10 |1.00 |
----------------------
-0.10 |0.00 |-0.10 |-1.00 |
----------------------
-0.10 |-0.10 |-0.10 |-0.10 |

Initial policy:
----------------------
  L    L    L       
----------------------
  L         D       
----------------------
  L    U    L    R  

values:
----------------------
0.80 |0.90 |1.00 |0.00 |
----------------------
0.70 |0.00 |0.90 |0.00 |
----------------------
0.60 |0.70 |0.80 |0.70 |

policy:
----------------------
  R    R    R       
----------------------
  U         U       
----------------------
  U    R    U    L  
